### Imports

In [ ]:
import pandas as pd
from pandas import DataFrame as DF
import matplotlib.pyplot as plt

from watea.watea_perfs import get_fleet_wise_energy_distribution
from core.plt_utils import plt_3d_df, plt_time_series_plotly
from watea.watea_constants import *

### Toutes les distributions

In [ ]:
df:DF = (
    get_fleet_wise_energy_distribution(False)
    .assign(soc=lambda df: df.index.get_level_values(2))
    .assign(temp=lambda df: df.index.get_level_values(1))
    .assign(odo=lambda df: df.index.get_level_values(0))
    .drop(columns=["duration"])
    .query("energy_added < 529")
    .eval("power = energy_added / sec_duration")
)

plt_3d_df(df, "soc", "power", "energy_added", color="odo", save_path=PATH_TO_FLEET_3D_DISTRIBUTION, size=4.5)

### Seul la distribution du "haut" est presente parmi toutes les ranges d'odometer

In [ ]:
plt_3d_df(df, "soc", "odo", "energy_added", color="odo", save_path=PATH_TO_FLEET_3D_DISTRIBUTION, size=4.5)

### Zoom sur la distribution la plus presente

In [ ]:
most_common_distribution_df:DF = (
    df
    .xs(15.0, level=1)
    .query("energy_added < 500 & energy_added > 300 & sec_duration < 900 & temp < 35")
)


plt_3d_df(most_common_distribution_df, "soc", "odo", "energy_added", color="power", save_path=PATH_TO_FLEET_3D_DISTRIBUTION, size=4.5)

### Nettoyage

In [ ]:
min_per_soc_and_odo_df = (
    most_common_distribution_df
    .groupby(level=[0, 1])["energy_added"]
    .quantile(0.2)
)
cleaned_df_mask = (
    most_common_distribution_df
    .groupby(level=[0, 1])["energy_added"]
    .transform(lambda energy_grp: energy_grp.gt(energy_grp.quantile(0.1)))
)
cleaned_df = (
    most_common_distribution_df
    .loc[cleaned_df_mask]
    .query("power < 4 & power > 1.5")
)

plt_3d_df(cleaned_df, "soc", "odo", "energy_added", color="sec_duration", save_path=PATH_TO_FLEET_3D_DISTRIBUTION, size=4.5)

### Comparaison entre la distribution du permier et du denier interval d'odometre

In [ ]:
keys = cleaned_df.index.get_level_values(level=0).unique()
first_odo_grp = cleaned_df.xs(key=keys[0], level=0)
LAST_KEY = -1
last_odo_grp = cleaned_df.xs(key=keys[LAST_KEY], level=0)


plt.scatter(first_odo_grp["soc"], first_odo_grp["energy_added"], color="green", label="0 to 3k odo", alpha=0.05)
plt.scatter(last_odo_grp["soc"], last_odo_grp["energy_added"], color="red", label="a lot of odo", alpha=0.1)
plt.show()

### Conclusion
Il n'y a pas bcp de difference ebtre la distribution 